# P5 : Pipline for Vehicle Detection and Tracking

This is the project report for the 5th and final Project for Term-1. Here, I will be going through each of the project rubric points as seprate sections and include explanation with example images to better represent a state of the code pipeline

In [1]:
# importing all the libraries that will be required for this project
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import time

from all_methods import *
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


** Prep for the pipeline**

Loading images in a list and creating a subset of first 3 images to test parameters and methods that is going to be used later in the notebook

In [2]:
# Reading vehicle and non-vehicle images and gettig a sense of the data
all_cars = glob.glob('training_samples/vehicles/*/*.png')
all_non_cars = glob.glob('training_samples/non-vehicles/*/*.png')
print(len(all_cars), len(all_non_cars))

8792 8968


In [3]:
def plotThreeImagesInRow(image1, image2, image3, title1='image1', title2='image2', title3='image3'):
    plt.clf() # clear any previous saved data
    
    # creating layout to place three image sided-by-side with their respective title
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image1, cmap='gray')
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(image2, cmap='gray')
    ax2.set_title(title2, fontsize=50)
    ax3.imshow(image3, cmap='gray')
    ax3.set_title(title3, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
    
def plotTwoImageInRow(image1, image2, title1='image1', title2='image2'):
    plt.clf()
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image1, cmap='gray')
    ax1.set_title(title1, fontsize=50)
    ax2.imshow(image2, cmap='gray')
    ax2.set_title(title2, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()
    
#def plotFeaturesHistorgram():

## 1. Histogram of Oriented Gradients (HOG)

**1.a : Explain how (and identify where in your code) you extracted HOG features from the training images. Explain how you settled on your final choice of HOG parameters.**

Following cells describe the method that extracts the HOG feature using the **hog()** method. The following values are set after experimenting with few values. I have chosen these values based on how closely they resemble the features in the original image. 

The first set of parameters captures the image features but at a top level. The second set of features captures the details in the image more granually. I will be experimenting with the classifier to see which of the two or something in between produces more accurate result keeping time delays less.

Final values for the parameter for the hog values can only be set after seeing how it made the classifier perform though.

**1.b : Describe how (and identify where in your code) you trained a classifier using your selected HOG features (and color features if you used them)**

In the following cells I will be writing the classifier and again testing them with different features combinations to increase the performance of the classifier.

I have used the SVM classifier. Following are the steps that goes into building and training the classifier:

1. **Extracting features : **I have used a combination of spatial, color histogram and HOG features for this classification. When tested for each of the features individually, I got 0.97 accuracy with only spatial features, 1.0 for histogram, 0.99 for HOG and 0.99 for all combined into one.
2. **Normalizing the features : **As suggested in the lessons, when combining different features to train the classifier there is always a possiblity that one feature might turn out to be more dominant than the other. That is why after I extracted the features for the car and non-car images, I am using **StandardScalar** to normalize my feature set.
3. **Train/Test splitting of data : **"I then created a sample of only 500 images from the list above and then divided that into training the testing set. I have kept the testing test as 20% of the total data.
3. **Create, train and test the classifier : **For classification I have created a SVM classifier and train and tested it out with different parameter yielding different accuracies. My final values for them in listed in the following cells

In [4]:
# parameters for extracting the features
colorspace = 'HLS'
orient = 9
pix_per_cell = 16  # 8 gives out accuracy of 0.99 and 4 gives out accuracy of 1.0
cell_per_block =4
hog_channel = 'ALL'

In [5]:
# extracting features for cars and not-car images
t=time.time()
car_features = extract_features(all_cars[:1], color_space=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(all_non_cars[:1], color_space=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()

hog (432,)
hog (432,)


In [6]:
print(round(t2-t, 2), 'Seconds to extract HOG features...')
print(len(car_features), len(notcar_features))

0.01 Seconds to extract HOG features...
1 1


In [11]:
# Stacking all the feature vectors for cars and non-car images.
# And then normalizing them all so that one feature doesnt become dominant in classification
X = np.vstack((car_features, notcar_features)).astype(np.float64)
X_scalar = StandardScaler().fit(X)
scaled_X = X_scalar.transform(X)
print(scaled_X.shape)

# creating the labels vectors for cars and non-cars images
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

(2, 3600)


In [ ]:
# spliting the sample data into training and testing set
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

In [ ]:
plt.plot(X[400])
plt.show()
plt.plot(scaled_X[400])
plt.show()

In [ ]:
# creating a linear SVM classifier to test the pipeline
svc = LinearSVC()
t1 = time.time() # start time
svc.fit(X_train, y_train)
t2 = time.time() # end time

print(round(t2-t, 2), 'Seconds to train SVC...')

# Checking the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

# Checking the prediction time for a single sample of 10 elements
t=time.time()
print('My SVC predicts: ', svc.predict(X_test))
print('For these',len(X_test), 'labels: ', y_test)
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', len(X_test),'labels with SVC')

## 2. Sliding Window Search ##

**2.a : Describe how (and identify where in your code) you implemented a sliding window search. How did you decide what scales to search and how much to overlap windows?
**


In [ ]:
# testing the sliding windows search
test_image_1 = mpimg.imread('test_images\\test6.jpg')
print(test_image_1.shape)

In [ ]:
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [400, 700] # Min and max in y to search in slide_window()

draw_image = np.copy(test_image_1)
windows = slide_window(test_image_1, y_start_stop=y_start_stop, xy_window=(96,96), xy_overlap=(0.5, 0.5))
car_windows = search_windows(test_image_1, windows=windows, clf=svc, scaler=X_scalar, color_space=colorspace, spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

In [ ]:
window_img = draw_boxes(test_image_1, car_windows, color=(0, 0, 255), thick=6)                    
plt.imshow(window_img)
plt.show()